<a href="https://colab.research.google.com/github/alvaroforner/Data_Engineer_test_AFD/blob/main/Q4_API_Jeff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import requests

In [ ]:
#Declaration of functions

def check_localdb(localdb):
  #Check if the DB exists, if not a new DB called Airlines_DB is created.
  try:
    df_db = pd.read_csv(localdb, sep=";")
    print("Database read successfully. You can proceed with the following steps")
  except:
    print("Database not found. A new database is generated.")
    df_db = database_generator(api)
    save_database(df_db, localdb)
    print("Database successfully generated. Name: Airlines_DB")
  return df_db 

#Function that generates the DB in case it is not found.
def database_generator(api):
  response = requests.get(api)
  response.json()["results"]
  airlines_db = pd.DataFrame(response.json()['results'])[['id', 'title',\
            'overview', 'popularity', 'release_date', 'vote_average', 'vote_count' ]]
  return airlines_db

#Function that saves in .CSV the last version of the DB.
def save_database(airlines_db, localdb):
  airlines_db.to_csv(localdb, sep=';', index = False, encoding='utf-8')

#Function that looks for the input parameter (ID) in API. In case that it does not exist restart the loop.
def buscador_id(api, input_parameter):
  response = requests.get(api)
  response.json()["results"]
  airlines_tmp = pd.DataFrame(response.json()['results'])[['id', 'title',\
            'overview', 'popularity', 'release_date', 'vote_average', 'vote_count' ]]
  fcond1 = airlines_tmp['id'] == input_parameter #checks if the id exists in the API.
  airlines_search = airlines_tmp[fcond1]
  if airlines_search.shape[0] != 0: #if it exists in the API the number of rows will be modistinct than 0
    print("Record found on the API")
    flag = True
    return airlines_search, flag
  else: 
    print("Idnetifier not located on the API")
    flag = False
    return airlines_search, flag

# Check if the register found on the API exists in local database, in case that it exists generates the mark "T", if not, generates the mark "F" in order to be able to update
#the attributes afterwards or add a new register.
def check_airline(df_db, input_parameter, airlines_search):
  def check_id(x, input_parameter):
    if x['id'] == input_parameter: #in case that the id already exists in the local DB, a mark is generated with "T" value
      res = "T"
    else:
      res = "F" # in case that the id does not exists in the local DB, a mark is generated with "F" value
    return res
  df_db['check'] = df_db.apply(lambda x: check_id(x, input_parameter), axis = 'columns')  
  fcond = df_db['check'] == "T"
  fcond2 = df_db['check'] == "F"
  
  if df_db[fcond].shape[0] != 0:
    print("[1] Identifier located in the Local Database")
    df_db = df_db[fcond2].copy()
    df_db.drop(['check'], axis=1, inplace = True)
    df_db = pd.concat([df_db, airline_search])
    print("[2] The register has been updated. ID: ", input_parameter)
    df_db.reset_index(drop =True, inplace = True)
  else: 
    print("[1] Identifier not found in the database")
    print("[2] New record is inserted in the local database. ID: ", input_parameter)
    df_db.drop(['check'], axis=1, inplace = True)
    df_db = pd.concat([df_db, airline_search])
    df_db.reset_index(drop =True, inplace = True)
  save_database(df_db, localdb)

In [ ]:
# Global variables
api =  "https://api.themoviedb.org/3/movie/top_rated?api_key=d73d7e5e4c1dda40808ef100a89f97c1&language=en-US&page=1"
localdb = 'airlines_db.txt'
flag_success = False


In [ ]:
# ***************
# Step I 
# ***************
# The Check_Localdb function is called, which checks if the DB exists locally, if it does, an ID is requested, if it does not exist, it creates the DB and requests an ID.
df_db = check_localdb(localdb)
while (flag_success == False):
  input_parameter = input("Enter an Airline ID: ")
  input_parameter = int(input_parameter)
# ***************
# Step II
# ***************
# The function buscador_id is called, which will search in the API the ID previously entered and will return 2 things: all the attributes associated with that ID and the flag 
# indicating if it has been able to locate it or not.

  buscador = buscador_id(api, input_parameter)
  airline_search = buscador[0]
  flag_search = buscador[1]
# ***************
# Step III 
# ***************

  if flag_search == True:
    check_airline(df_db, input_parameter, airline_search)
    flag_success = True
  else: pass


Database read successfully. You can proceed with the following steps
Enter an Airline ID: 31000
Idnetificador no localizado en la plataforma de airlines.
Enter an Airline ID: 98
Idnetificador no localizado en la plataforma de airlines.
Enter an Airline ID: 990
Idnetificador no localizado en la plataforma de airlines.
Enter an Airline ID: 000
Idnetificador no localizado en la plataforma de airlines.
Enter an Airline ID: 278
Registro localizado en la plataforma de airlines
[1] Identificador localizado en la Base de datos local
[2] El registro se ha actualizado. ID:  278


''

In [ ]:
df_db



,id,title,overview,popularity,release_date,vote_average,vote_count,check
0,19404,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...",23.036,1995-10-20,8.7,3679,F
1,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",63.339,1972-03-14,8.7,16071,F
2,424,Schindler's List,The true story of how businessman Oskar Schind...,43.257,1993-11-30,8.6,12842,F
3,240,The Godfather: Part II,In the continuing saga of the Corleone crime f...,52.401,1974-12-20,8.6,9689,F
4,372754,Dou kyu sei – Classmates,"Rihito Sajo, an honor student with a perfect s...",9.903,2016-02-20,8.6,230,F
5,667257,Impossible Things,A widow who is tormented by the memory of her ...,33.137,2021-06-17,8.6,242,F
6,129,Spirited Away,"A young girl, Chihiro, becomes trapped in a st...",88.845,2001-07-20,8.5,12922,F
7,372058,Your Name.,High schoolers Mitsuha and Taki are complete s...,207.313,2016-08-26,8.5,8778,F
8,696374,Gabriel's Inferno,An intriguing and sinful exploration of seduct...,10.954,2020-05-29,8.5,2248,F
9,533514,Violet Evergarden: The Movie,As the world moves on from the war and technol...,18.903,2020-09-18,8.5,235,F
